In [139]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from preprocessing import preprocess

In [140]:
data = pd.read_csv("train.csv", index_col=0)
data_plus = pd.read_csv("training_extra.csv", index_col=0)
data = pd.concat([data, data_plus], axis=0)

In [141]:
data.tail()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
4194313,Nike,Canvas,NaN,3.0,Yes,Yes,Messenger,Blue,28.098120,104.74460
4194314,Puma,Leather,Small,10.0,Yes,Yes,Tote,Blue,17.379531,122.39043
4194315,Jansport,Canvas,Large,10.0,No,No,Backpack,Red,17.037708,148.18470
4194316,Puma,Canvas,NaN,2.0,No,No,Backpack,Gray,28.783339,22.32269
4194317,Under Armour,Polyester,Medium,2.0,Yes,No,Backpack,Blue,23.076169,107.61199


In [142]:
data = data[:150000]

In [143]:
data = preprocess(data)
data.head()

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEmin', 'Brand_TEmax', 'Brand_TEskew', 'Material_TEmean',
       'Material_TECount', 'Material_TESTD', 'Material_TEVAR',
       'Material_TEnunique', 'Material_TEmedian', 'Material_TEmin',
       'Material_TEmax', 'Material_TEskew', 'Size_TEmean', 'Size_TECount',
       'Size_TESTD', 'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian',
       'Size_TEmin', 'Size_TEmax', 'Size_TEskew', 'Style_TEmean',
       'Style_TECount', 'Style_TESTD', 'Style_TEVAR', 'Style_TEnunique',
       'Style_TEmedian', 'Style_TEmin', 'Style_TEmax', 'Style_TEskew',
       'Color_TEmean', 'Color_TECount', 'Color_TESTD', 'Color_TEVAR',
       'Color_TEnuniq

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,...,Style_TEskew,Color_TEmean,Color_TECount,Color_TESTD,Color_TEVAR,Color_TEnunique,Color_TEmedian,Color_TEmin,Color_TEmax,Color_TEskew
0,Jansport,Leather,Medium,7.0,1.0,0.0,Tote,Black,11.611723,112.15875,...,0.048537,80.555044,23186,39.242215,1539.951402,17816,79.334665,15.0,150.0,0.079296
1,Jansport,Canvas,Small,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056,...,0.032795,82.351278,23094,39.018596,1522.450851,17857,82.218545,15.0,150.0,0.012331
2,Under Armour,Leather,Small,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320,...,0.032795,80.868385,23635,39.182374,1535.258467,18179,80.456350,15.0,150.0,0.051953
3,Nike,Nylon,Small,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793,...,0.032795,82.351278,23094,39.018596,1522.450851,17857,82.218545,15.0,150.0,0.012331
4,Adidas,Canvas,Medium,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312,...,0.032795,82.351278,23094,39.018596,1522.450851,17857,82.218545,15.0,150.0,0.012331


In [144]:
data.dtypes

Brand                            object
Material                         object
Size                             object
Compartments                    float64
Laptop Compartment              float64
Waterproof                      float64
Style                            object
Color                            object
Weight Capacity (kg)            float64
Price                           float64
Laptop Compartment_Missing        int32
Waterproof_Missing                int32
Weight Capacity (kg)_Missing      int32
Brand_TEmean                    float64
Brand_TECount                     int64
Brand_TESTD                     float64
Brand_TEVAR                     float64
Brand_TEnunique                   int64
Brand_TEmedian                  float64
Brand_TEmin                     float64
Brand_TEmax                     float64
Brand_TEskew                    float64
Material_TEmean                 float64
Material_TECount                  int64
Material_TESTD                  float64


---

FIRST SANDBOX - testing before implementing in preprocessing function

In [145]:
numerical_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(exclude='number').columns

In [146]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEmin', 'Brand_TEmax', 'Brand_TEskew', 'Material_TEmean',
       'Material_TECount', 'Material_TESTD', 'Material_TEVAR',
       'Material_TEnunique', 'Material_TEmedian', 'Material_TEmin',
       'Material_TEmax', 'Material_TEskew', 'Size_TEmean', 'Size_TECount',
       'Size_TESTD', 'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian',
       'Size_TEmin', 'Size_TEmax', 'Size_TEskew', 'Style_TEmean',
       'Style_TECount', 'Style_TESTD', 'Style_TEVAR', 'Style_TEnunique',
       'Style_TEmedian', 'Style_TEmin', 'Style_TEmax', 'Style_TEskew',
       'Color_TEmean', 'Color_TECount', 'Color_TESTD', 'Color_TEVAR',
       'Color_TEnuniq

In [147]:
categorical_columns

Index(['Brand', 'Material', 'Size', 'Style', 'Color'], dtype='object')

END OF THE FIRST SANDBOX

---

In [148]:
print(f"{(len(numerical_columns) + len(categorical_columns))} = {len(data.columns)}")

58 = 58


In [149]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEmin', 'Brand_TEmax', 'Brand_TEskew', 'Material_TEmean',
       'Material_TECount', 'Material_TESTD', 'Material_TEVAR',
       'Material_TEnunique', 'Material_TEmedian', 'Material_TEmin',
       'Material_TEmax', 'Material_TEskew', 'Size_TEmean', 'Size_TECount',
       'Size_TESTD', 'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian',
       'Size_TEmin', 'Size_TEmax', 'Size_TEskew', 'Style_TEmean',
       'Style_TECount', 'Style_TESTD', 'Style_TEVAR', 'Style_TEnunique',
       'Style_TEmedian', 'Style_TEmin', 'Style_TEmax', 'Style_TEskew',
       'Color_TEmean', 'Color_TECount', 'Color_TESTD', 'Color_TEVAR',
       'Color_TEnuniq

In [150]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
import xgboost as xgb

In [151]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEmin', 'Brand_TEmax', 'Brand_TEskew', 'Material_TEmean',
       'Material_TECount', 'Material_TESTD', 'Material_TEVAR',
       'Material_TEnunique', 'Material_TEmedian', 'Material_TEmin',
       'Material_TEmax', 'Material_TEskew', 'Size_TEmean', 'Size_TECount',
       'Size_TESTD', 'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian',
       'Size_TEmin', 'Size_TEmax', 'Size_TEskew', 'Style_TEmean',
       'Style_TECount', 'Style_TESTD', 'Style_TEVAR', 'Style_TEnunique',
       'Style_TEmedian', 'Style_TEmin', 'Style_TEmax', 'Style_TEskew',
       'Color_TEmean', 'Color_TECount', 'Color_TESTD', 'Color_TEVAR',
       'Color_TEnuniq

In [152]:
ohe_columns = [
    "Brand",
    "Material",
    "Size",
    #"Compartments",
    "Style",
    "Color"
]

columns_to_scale = [
    "Compartments",
    "Weight Capacity (kg)"
] + [col for col in data.columns if "TE" in col]

In [153]:
columns_to_scale

['Compartments',
 'Weight Capacity (kg)',
 'Brand_TEmean',
 'Brand_TECount',
 'Brand_TESTD',
 'Brand_TEVAR',
 'Brand_TEnunique',
 'Brand_TEmedian',
 'Brand_TEmin',
 'Brand_TEmax',
 'Brand_TEskew',
 'Material_TEmean',
 'Material_TECount',
 'Material_TESTD',
 'Material_TEVAR',
 'Material_TEnunique',
 'Material_TEmedian',
 'Material_TEmin',
 'Material_TEmax',
 'Material_TEskew',
 'Size_TEmean',
 'Size_TECount',
 'Size_TESTD',
 'Size_TEVAR',
 'Size_TEnunique',
 'Size_TEmedian',
 'Size_TEmin',
 'Size_TEmax',
 'Size_TEskew',
 'Style_TEmean',
 'Style_TECount',
 'Style_TESTD',
 'Style_TEVAR',
 'Style_TEnunique',
 'Style_TEmedian',
 'Style_TEmin',
 'Style_TEmax',
 'Style_TEskew',
 'Color_TEmean',
 'Color_TECount',
 'Color_TESTD',
 'Color_TEVAR',
 'Color_TEnunique',
 'Color_TEmedian',
 'Color_TEmin',
 'Color_TEmax',
 'Color_TEskew']

In [154]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
std = StandardScaler()

In [155]:
#submission_on = False
#preprocess_transformer = FunctionTransformer(
#    preprocess, kw_args={'submission': submission_on}
#)

In [156]:
column_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoding", ohe, ohe_columns),
        ("Standardization", std, columns_to_scale)
    ],
    remainder="passthrough",
    n_jobs=-1,
    verbose=True,
    verbose_feature_names_out=False
).set_output(transform="pandas")

In [157]:
pipeline = Pipeline(
    [
        #("Preprocessing", preprocess_transformer),
        ("ColumnTransformer", column_transformer),
        #("Model", xgb.XGBRegressor())
    ]
)

In [158]:
X = data.drop(columns='Price')
y = data['Price']

In [159]:
X = pipeline.fit_transform(X)

In [160]:
X.head(3)

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_TEnunique,Color_TEmedian,Color_TEmin,Color_TEmax,Color_TEskew,Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.085227,-1.281725,0.0,0.0,1.448434,1.0,0.0,0,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,1.0,1.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.056341,-0.333379,0.0,0.0,0.489767,1.0,0.0,0,0,0


---
SECOND SANDBOX

END OF THE SECOND SANDBOX

---

In [161]:
# need to find out again the best combinations

import json

with open('best_combinations_22_02.json', 'r') as json_file:
    relevant_combinations = json.load(json_file)

combinations_to_add = [] #list(relevant_combinations.keys())[:50]

for combination in combinations_to_add:
    separation = combination.split("*")
    first_col = separation[0]
    second_col = separation[1]
    X[combination] = X[first_col] * X[second_col]

In [162]:
X.head()

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_TEnunique,Color_TEmedian,Color_TEmin,Color_TEmax,Color_TEskew,Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.085227,-1.281725,0.0,0.0,1.448434,1.0,0.0,0,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,1.0,1.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.056341,-0.333379,0.0,0.0,0.489767,1.0,0.0,0,0,0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,1.0,0.0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,1.0,1.0,0,0,0


In [163]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

KF = KFold(n_splits=5, shuffle=False)

In [164]:
#model = LinearRegression()
model = xgb.XGBRegressor()

---

CROSS VAL

In [165]:
#cv_results = cross_validate(model, X, y, cv=KF, verbose=3, return_train_score=True, scoring='neg_root_mean_squared_error', n_jobs=-1)
#print(cv_results['train_score'].mean())
#print(cv_results['test_score'].mean())

In [166]:
# only with the first twenty : -38.880299988987474

In [167]:
model = xgb.XGBRegressor()
model.fit(X, y, verbose=3)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

---

GRID SEARCH

In [168]:
from sklearn.model_selection import GridSearchCV

In [169]:
xgboost_param_grid = {
    'n_estimators': [100, 300],  
    'max_depth': [6, 10],  
    'learning_rate': [0.01, 0.2],
    'subsample': [0.85],
    'colsample_bytree': [0.6, 0.8],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [0.1, 1.0],
    'eval_metric': ['logloss'],
    'booster': ['gbtree'],
    'min_child_weight': [1, 3]
}

xgboost_grid = GridSearchCV(
    estimator=model,
    param_grid=xgboost_param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=True,
    n_jobs=-1
)

In [170]:
#xgboost_grid.fit(X, y)
#print(f"Best XGBoost Params: {xgboost_grid.best_params_}")
#print(f"Best model's score : {xgboost_grid.best_score_}")
#xgboost_best_model = xgboost_grid.best_estimator_

---

DATA TEST

In [171]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [172]:
test = preprocess(test, submission=True)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Laptop Compartment_Missing,...,Style_TEskew,Color_TEmean,Color_TECount,Color_TESTD,Color_TEVAR,Color_TEnunique,Color_TEmedian,Color_TEmin,Color_TEmax,Color_TEskew
id,,,,,,,,,,,,,,,,,,,,,
300000,Puma,Leather,Small,2.0,0.0,0.0,Tote,Green,20.671147,0,...,0.048537,82.351278,23094.0,39.018596,1522.450851,17857.0,82.218545,15.0,150.0,0.012331
300001,Nike,Canvas,Medium,7.0,0.0,1.0,Backpack,Green,13.564105,0,...,0.035718,82.351278,23094.0,39.018596,1522.450851,17857.0,82.218545,15.0,150.0,0.012331
300002,Adidas,Canvas,Large,9.0,0.0,1.0,Messenger,Blue,11.809799,0,...,0.032795,81.953437,24090.0,38.613940,1491.036379,18564.0,82.446105,15.0,150.0,-0.004556
300003,Adidas,Nylon,Large,1.0,1.0,0.0,Messenger,Green,18.477036,0,...,0.032795,82.351278,23094.0,39.018596,1522.450851,17857.0,82.218545,15.0,150.0,0.012331
300004,Inconnu,Nylon,Large,2.0,1.0,1.0,Tote,Black,9.907953,0,...,0.048537,80.555044,23186.0,39.242215,1539.951402,17816.0,79.334665,15.0,150.0,0.079296


In [173]:
test = pipeline.transform(test)
test.head()

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_TEnunique,Color_TEmedian,Color_TEmin,Color_TEmax,Color_TEskew,Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
id,,,,,,,,,,,,,,,,,,,,,
300000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,0.0,0.0,0,0,0
300001,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,0.0,1.0,0,0,0
300002,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.206488,1.348890,0.0,0.0,-1.491413,0.0,1.0,0,0,0
300003,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.069237,1.156496,0.0,0.0,-0.899340,1.0,0.0,0,0,0
300004,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.085227,-1.281725,0.0,0.0,1.448434,1.0,1.0,0,0,0


In [174]:
for combination in combinations_to_add:
    separation = combination.split("*")
    first_col = separation[0]
    second_col = separation[1]
    test[combination] = test[first_col] * test[second_col]

In [175]:
preds = pd.concat([pd.DataFrame(test.index), pd.DataFrame(model.predict(test))], axis=1, ignore_index=True)
preds.columns = ['id', 'Price']
print(len(preds))
preds.head()

200000


,id,Price
0,300000,81.895798
1,300001,80.247101
2,300002,81.897186
3,300003,78.016693
4,300004,77.370888


In [176]:
preds.to_csv('submission.csv', index=False)